In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

In [3]:
# CNN -> BatchNorm -> RelU (Common Sturcture)
# x10

class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size = 3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding='same')
        self.bn = layers.BatchNormalization()
        
    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training = training)
        x = tf.nn.relu(x)
        return x

In [4]:
class ResBlock(layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.cnn1 = CNNBlock(channels[0])
        self.cnn2 = CNNBlock(channels[1])
        self.cnn3 = CNNBlock(channels[2])
        self.pooling = layers.MaxPool2D()
        self.identity_mapping = layers.Conv2D(channels[1], 3, padding='same')
        
    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(
            x + self.identity_mapping(input_tensor), training=training
        )
        
        return self.pooling(x)

In [5]:
class ResNet_Like(keras.Model):
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__()
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([128, 128, 256])
        self.block3 = ResBlock([128, 256, 512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)
        
    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x)
        return self.classifier(x)
    
    def model(self):
        x = keras.Input(shape=(28,28,1))
        return keras.Model(inputs=[x], outputs=self.call(x))

In [6]:
# model = keras.Sequential([
#     CNNBlock(32),
#     CNNBlock(64),
#     CNNBlock(128),
#     layers.Flatten(),
#     layers.Dense(10),
# ])

In [7]:
model = ResNet_Like(num_classes=10)

In [8]:
print(model.model().summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
res_block (ResBlock)         (None, 14, 14, 64)        28896     
_________________________________________________________________
res_block_1 (ResBlock)       (None, 7, 7, 256)         592512    
_________________________________________________________________
res_block_2 (ResBlock)       (None, 3, 3, 512)         2364032   
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                5130      
Total params: 2,990,570
Trainable params: 2,987,498
Non-trainable params: 3,072
_______________________________________________

In [9]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

In [11]:
#model.fit(x_train, y_train, batch_size=64, epochs=1, verbose=2)

In [12]:
#model.evaluate(x_test, y_test, bath_size=64, verbose=2)